<a href="https://colab.research.google.com/github/Galzi1/DataDiscoveryNNProj/blob/master/project_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Knowledge Data Discovery and Neural Networks : Final Project

In this notebook we will prepare data and run several algorithms for classification

# 1. Imports

In [45]:
# add more packages in this section
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# 2. data preperation

In [46]:
# data = pd.read_csv("data/adult.data", header = None)
data = pd.read_csv("https://raw.githubusercontent.com/Galzi1/DataDiscoveryNNProj/master/data/adult.data", header = None)
cols = ['age', 'workclass', 'fnlwgt','education','education_num','marital_status','occupation','relationship','race','sex','capital_gain', 'capital_loss'
    ,'hours_per_week','native_country','y']
data.columns = cols

## 2.1 Categorical feature handling
* Name two machine learning algorithms that can deal with categorical features without special handling?

special handling = one hot encoding exc..

#### YOUR VERBAL SOLUTION HERE
* Naive Bayes (explain some)
* Some Tree-based algorithm (specify one and explain)


#### END YOUR VERBAL SOLUTION HERE

## 2.2 One hot encoding 

* Remove the y column from the data variable and save it to the variable y
* Transform the categorical columns to one hot encoding

You may find get_dummies function in pandas useful

In [47]:
### YOUR CODE HERE
y = data['y']
X = data.drop(['y'], axis=1)

obj_cols = list(X.select_dtypes(include=['object']).columns)

X = pd.get_dummies(X, columns=obj_cols, drop_first=True)

X.head()


### END YOUR CODE

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,workclass_ Without-pay,education_ 11th,education_ 12th,education_ 1st-4th,education_ 5th-6th,education_ 7th-8th,education_ 9th,education_ Assoc-acdm,education_ Assoc-voc,education_ Bachelors,education_ Doctorate,education_ HS-grad,education_ Masters,education_ Preschool,education_ Prof-school,education_ Some-college,marital_status_ Married-AF-spouse,marital_status_ Married-civ-spouse,marital_status_ Married-spouse-absent,marital_status_ Never-married,marital_status_ Separated,marital_status_ Widowed,occupation_ Adm-clerical,occupation_ Armed-Forces,occupation_ Craft-repair,occupation_ Exec-managerial,occupation_ Farming-fishing,...,native_country_ Canada,native_country_ China,native_country_ Columbia,native_country_ Cuba,native_country_ Dominican-Republic,native_country_ Ecuador,native_country_ El-Salvador,native_country_ England,native_country_ France,native_country_ Germany,native_country_ Greece,native_country_ Guatemala,native_country_ Haiti,native_country_ Holand-Netherlands,native_country_ Honduras,native_country_ Hong,native_country_ Hungary,native_country_ India,native_country_ Iran,native_country_ Ireland,native_country_ Italy,native_country_ Jamaica,native_country_ Japan,native_country_ Laos,native_country_ Mexico,native_country_ Nicaragua,native_country_ Outlying-US(Guam-USVI-etc),native_country_ Peru,native_country_ Philippines,native_country_ Poland,native_country_ Portugal,native_country_ Puerto-Rico,native_country_ Scotland,native_country_ South,native_country_ Taiwan,native_country_ Thailand,native_country_ Trinadad&Tobago,native_country_ United-States,native_country_ Vietnam,native_country_ Yugoslavia
0,39,77516,13,2174,0,40,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,50,83311,13,0,0,13,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,38,215646,9,0,0,40,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,53,234721,7,0,0,40,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,28,338409,13,0,0,40,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## 2.3 Train test split

* Convert the y column - replace <=50K with 0 and >50K with 1 
* Split the dataset into train and test set (use 15% for the test set)

at the end, make sure you have the following variables:

* X
* y
* X_train
* X_test
* y_train
* y_test

You may find sklearn train_test_split useful

In [48]:
### YOUR CODE HERE
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)




### END YOUR CODE

## 2.3 Numeric feature normalization

* Scale the numeric features to to have zero mean (z score normalization)

- Don't scale the boolean features

You may find sklearn StandardScaler useful

In [52]:
### YOUR CODE HERE
from sklearn import preprocessing

int_cols = list(X.select_dtypes(include=['int64']).columns)
print(int_cols)

X[int_cols] = preprocessing.StandardScaler().fit_transform(X[int_cols])

### END YOUR CODE

['age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week']


# 3 Models

Create a function `cv(x, y, model)` (cv stands for cross validation) that gets a model (sklearn classifier) and the data.  
The function should fit the model using k fold cross validation with k = 5, and print the 'roc_auc' on each fold (which is the scoring parameter for the cross val_score function).


[Computing cross-validated metrics](https://scikit-learn.org/stable/modules/cross_validation.html)


In [ ]:
## YOUR CODE HERE
from sklearn.model_selection import cross_val_score
from sklearn import metrics

def cv(x, y, model):
  print(cross_val_score(model, X, y, cv=5, scoring='roc_auc'))




## END YOUR CODE

## 3.1 Baseline model - logistic regression
Our first algorithm will be logistic regression, since it's always nice to know how well can we do with a simple algorithm.
* Should we use class_weight = 'balanced' in sklearn logistic regression? why?
* Check how the results differ with and without the 'balanced' parameter - use `cv(x, y, model)` for that purpose, explain your answer
* Fit a model on X_train, y_train and plot the precision recall curve on the test data.
* If we would plot this curve on each fold (of the 5 folds in the cross validation), would we get exactly the same plot?

Do not change other hyperparameters

[precision recall curve](https://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html)

In [ ]:
## YOUR CODE HERE







## END YOUR CODE

#### YOUR VERBAL SOLUTION HERE



#### END YOUR VERBAL SOLUTION HERE

## 3.2 ANN
ANN (MLPClassifier in sklearn):

* Fit a model on X_train, y_train and print the AUC and the Log Loss on the train and test data.
* Explain your results - are they better or worse than the baseline? try to explain why

Do not change hyperparameters

In [ ]:
### YOUR CODE HERE







### END YOUR CODE

#### YOUR VERBAL SOLUTION HERE



#### END YOUR VERBAL SOLUTION HERE

## 3.3 Random forest
Random Forest classifier:

* Fit a model on X_train, y_train and print the AUC and the Log Loss on the train and test data.
* Explain your results - are they better or worse than the baseline? try to explain why

Do not change hyperparameters


In [ ]:
### YOUR CODE HERE
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

clf = RandomForestClassifier()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
print(metrics.auc(fpr, tpr))







### END YOUR CODE

ValueError: ignored

#### YOUR VERBAL SOLUTION HERE



#### END YOUR VERBAL SOLUTION HERE

## 3.4 Confusion matrix
Plot/print the confusion matrix of the random forest model on the test data

Explain shortly your results

In [ ]:
### YOUR CODE HERE
from sklearn.metrics import plot_confusion_matrix

class_names = y.unique()

np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
titles_options = [("Confusion matrix, without normalization", None),
                  ("Normalized confusion matrix", 'true')]
for title, normalize in titles_options:
    disp = plot_confusion_matrix(clf, X_test, y_test,
                                 display_labels=class_names,
                                 cmap=plt.cm.Blues,
                                 normalize=normalize)
    disp.ax_.set_title(title)

    print(title)
    print(disp.confusion_matrix)

plt.show()


### END YOUR CODE

NotFittedError: ignored

#### YOUR VERBAL SOLUTION HERE



#### END YOUR VERBAL SOLUTION HERE

## 3.4 Global feature importance of the random forest and SHAP

* Plot the global feature importance of the features - use SHAP for this purpsose
* Choose two samples from the dataset and plot/print the local explanations for this samples. Explain which features
are important
* Explain in few words how this feature importance is calculated

**SHAP can be very slow on the random forest model. Thus, train another model with shallow trees (depth <7 for example) and you can also compute the SHAP values on a small data set. If it is yet very slow you, try to fit a different classifier for this purpose (MLP for example or GradientBoostingClassifier). You can also discard the binary features (which derived from the categorical features) for this purpose and stay only with the numerical ones.**


In [ ]:
### YOUR CODE HERE








### END YOUR CODE

#### YOUR VERBAL SOLUTION HERE



#### END YOUR VERBAL SOLUTION HERE

## 4 Clustering
Imagine that we don't really know the true labels -> we need to use unsupervised machine learning.

* Perform k means on X_train with k = 2. Is our clusters represent rich and poor people (does one cluster represent 'rich' ('>= 50k') people and the other 'poor' people)?

* Do the same with dbscan, understand how many clusters did you get and the proportion of 'rich' and 'poor' people in each cluster.

* Plot the clusters using pca (only for the kmeans). Are they seperated in the PCA dimension?
 
This question is more open minded and you can (recomended) use graphs that explain how well did the clustering work. Did it work well?? 

In [53]:
print(list(X_train.loc[pd.isna(X_train["age"]), :].index))
print(X_train.loc[28956, :])
# print(X_train.isna().sum())
# print("########################")
# # 2)
# for c in X_train.columns:
#   if not np.issubdtype(X_train[c].dtype, np.number):
#     print(f'Column = {c}; unique values = {sorted(X_train[c].unique())}')
# print("########################")
# # 3)
# for c in X_train.columns:
#   if not np.issubdtype(X_train[c].dtype, np.number):
#   print(f'Column = {c}; Number of "?" = {X_train[X_train[c] == " ?"].shape[0]}')
# print("########################")

[]
age                                    35
fnlwgt                             101509
education_num                          10
capital_gain                            0
capital_loss                            0
                                    ...  
native_country_ Thailand                0
native_country_ Trinadad&Tobago         0
native_country_ United-States           1
native_country_ Vietnam                 0
native_country_ Yugoslavia              0
Name: 28956, Length: 100, dtype: int64


In [ ]:
### YOUR CODE HERE
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2)
kmeans.fit(X_train)
y_kmeans = kmeans.predict(X_train)

plt.scatter(X_train[:, 0], X_train[:, 1], c=y_kmeans, s=50, cmap='viridis')

centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);

plt.show()

### END YOUR CODE

ValueError: ignored

#### YOUR VERBAL SOLUTION HERE



#### END YOUR VERBAL SOLUTION HERE